# Project 3 : Collecting children books data from amazon
by : Modhi Almannaa

-----
## Introduction

In this project i'm gone to scraping data from Amazon, it's largest selling platform, it provide all the information that helps the customer, to find the best products, By reviewing all the information the customer needs to know before buying the product.
Here we focused only on children books.

### Contents:
- [Import important libraries](#Import-important-libraries)
- [Collect links of pages](#Collect-links-of-pages)
- [Collect important data](#Collect-important-data)
- [Data dictionary](#Data-dictionary)
- [Clean missing values](#Clean-missing-values)
- [Fix incorrect data types](#Fix-incorrect-data-types)
- [saving the data and upload it on kaggel](#saving-the-data-and-upload-it-on-kaggel)
- [Conclusions](#Conclusions)

---
## Import important libraries

In [1]:
import re #Regular expression
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup , NavigableString, Tag
from time import sleep
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

---
## Collect links of pages

In [3]:
m=[]
m.append('https://www.amazon.com/s?i=stripbooks&rh=n%3A4%2Cp_72%3A1250221011&pf_rd_i=4&pf_rd_p=f92327d7-7b1b-5fb4-82be-5400739bec4f&pf_rd_r=DD6Z1HDJS8TPFG2MAF2E&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&qid=1610894281&ref=sr_pg_1')
for i in range (2,76):
    next_page='https://www.amazon.com/s?i=stripbooks&rh=n%3A4%2Cp_72%3A1250221011&page={}&pf_rd_i=4&pf_rd_p=f92327d7-7b1b-5fb4-82be-5400739bec4f&pf_rd_r=DD6Z1HDJS8TPFG2MAF2E&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&qid=1611149061&ref=sr_pg_{}'.format(i,i)
    m.append(next_page)

In [3]:
len(m)

75

---
## Collect important data

In [4]:
name=[]
part_of=[]
Author =[]
rating=[]
no_of_ratings =[]
age=[]
publication_date=[]
price =[]
price_befor =[]
cover_type=[]
links=[]
description=[]
Product_details =[]
best_seller=[]

In [5]:
driver = webdriver.Chrome(executable_path='chromedriver/chromedriver')

for k in range(0,75):
#         driver = webdriver.Chrome(executable_path='chromedriver/chromedriver')
    driver.get(m[k])
    sleep(5) 
    html = driver.page_source
    soup1 = BeautifulSoup(html,'html')
    items = soup1.find_all('div' , attrs={'data-component-type':"s-search-result" ,'class':"s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col sg-col-12-of-16" })
# for name
    for i in items:
        name.append(i.find('span', class_="a-size-medium a-color-base a-text-normal").string)
#for series
    for i in items:
        try:
            part_of.append(str(i.find('div', class_="a-section a-spacing-none").find_next('div', class_="a-row a-size-base a-color-secondary").find('a' , class_="a-link-normal a-text-normal").text).replace('\n',''))
        except:
             part_of.append(np.nan)
#for Author
    for i in items:
        try:
            try: 
                Author.append(str(i.find('div', class_="a-row a-size-base a-color-secondary").find('a' , class_="a-size-base a-link-normal").text).replace('\n',''))
            except:
                a = i.find('div', class_="a-row a-size-base a-color-secondary").find_all('span' , attrs={'dir':"auto"})[1].text

                if a == ' | ' :
                    Author.append(i.find('div', class_="a-row a-size-base a-color-secondary").find_all('span' , attrs={'dir':"auto"})[3].text)
                else:
                    Author.append(a)                
        except:
             Author.append(np.nan)
#for rating
    for i in items:
        try:
            rating.append(str(i.find('a', class_="a-popover-trigger a-declarative").text))
        except:
             rating.append(np.nan)
# for no_of_ratings 
    for i in items:
        try:
            no_of_ratings.append(i.find('div', attrs= {'class':"a-row a-size-small"}).find('a', attrs= {'class':"a-link-normal"}).find('span', attrs= {'class':"a-size-base"}).text)
        except:
             no_of_ratings.append(np.nan)

# for age
    for i in items:
        try:
            age.append(i.find('div', attrs={'class':"sg-col-4-of-12 sg-col-4-of-16 sg-col sg-col-8-of-20"}).text.replace('\n',''))
        except:
             age.append(np.nan)                
# for publication_date
    for i in items:
        try:
            publication_date.append(i.find('div', attrs={'class':"a-row a-size-base a-color-secondary"}).find('span', attrs={'class':"a-size-base a-color-secondary a-text-normal"}).text.replace('\n',''))
        except:
             publication_date.append(np.nan)                
# for price
    for i in items:
        try:
            price.append(i.find('span',class_="a-offscreen").text)# ,i.find('span', attrs= {'class':"a-price-fraction"}).text)
        except:
             price.append(np.nan)

#for price_befor =[]
    for i in items:
        try:
            price_befor.append(i.find('span',class_="a-price a-text-price").find('span', attrs= {'class':"a-offscreen"}).text)
        except:
             price_befor.append(np.nan)                
# for cover_type=[]
    for i in items:
        try:
            cover_type.append(i.find('div', class_="a-section a-spacing-none a-spacing-top-small").find('div', class_="a-row a-size-base a-color-base").find('a' ,class_="a-size-base a-link-normal a-text-bold").text.replace('\n',''))
        except:
             cover_type.append(np.nan)                
#for links to get describtion
    for i in items:
        try:
            links.append('https://www.amazon.com'  + i.find('a' , class_="a-link-normal a-text-normal")['href'])
        except:
             links.append(np.nan)                


In [13]:
# to collect description & Product_details
driver = webdriver.Chrome(executable_path='chromedriver/chromedriver')
for h in range(len(links)):
    driver.get(links[h])
    sleep(5) 
    html = driver.page_source
    soup1 = BeautifulSoup(html,'html')

  # for description 
    try:
        description.append(soup1.find('div', attrs={ 'id':"bookDescription_feature_div" , 'class':"celwidget" , 'data-feature-name':"bookDescription"}).find('noscript').text)
    except:
        description.append(np.nan)   
 # for Product_details
    try:
        Product_details.append(soup1.find('div', attrs={'id':"detailBullets_feature_div" , 'class':"celwidget" ,  'data-feature-name':"detailBullets"}).text)
    except:
        Product_details.append(np.nan)
 # for best_seller
    try:
        best_seller.append(soup1.find('div', attrs={'id':"zeitgeistBadge_feature_div" , 'class':"celwidget" ,  'data-feature-name':"zeitgeistBadge"}).find('span', class_="cat-link").text)
    except:
        best_seller.append(np.nan)

In [18]:
print(len(name),len(part_of),len(Author),len(rating),len(no_of_ratings),len(age),len(publication_date),len(price),len(price_befor),len(cover_type),len(links),len(description),len(Product_details))

1200 1200 1200 1200 1200 1200 1200 1200 1200 1200 1200 1200 1200


In [11]:
# name.clear()
# part_of.clear()
# Author.clear()
# rating.clear()
# no_of_ratings.clear()
# age.clear()
# publication_date.clear()
# price.clear()
# price_befor.clear()
# cover_type.clear()
# links.clear()
# description.clear()
# Product_details.clear()

In [17]:
print(name[16],
    part_of[16],'----',
    Author[16],'----',
    rating[16],
    no_of_ratings[16],
    age[16],
    publication_date[16],
    price[16],
    price_befor[16],
    cover_type[16],
    links[16],
    description[16],
    Product_details[16],
     best_seller[16])

School Zone - Big Preschool Workbook - Ages 3 to 5, Colors, Shapes, Numbers 1-10, Early Math, Alphabet, Pre-Writing, Phonics, Following Directions, and More (School Zone Big Workbook Series) nan ---- School Zone ---- 4.8 out of 5 stars 42,288 Ages: 3 - 5 years Jun 7, 2018 $8.19 $12.99 Paperback https://www.amazon.com/School-Zone-Preschool-Pre-Writing-Pre-Reading/dp/0887431453/ref=sr_1_17?dchild=1&pf_rd_i=4&pf_rd_p=f92327d7-7b1b-5fb4-82be-5400739bec4f&pf_rd_r=DD6Z1HDJS8TPFG2MAF2E&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&qid=1611149061&refinements=p_72%3A1250221011&s=books&sr=1-17 
School Zone products shipped and sold by Amazon are genuine and authentic. Products sold by other sellers may be of lesser quality and are not guaranteed by School Zone. Find our complete line of educational resources at Amazon.com/SchoolZonePublishing. BOOK FEATURES  320 pages: 319 activity pages, 1 award certificate For ages 3-5 7.75" x 10.75" pages with durable glossy cover Book has 5 sections with Par

In [22]:
child_books = pd.DataFrame({'Name' : name,
                            'Series':part_of,
                            'Author':Author,
                            'Rating':rating,
                            'No_of_Ratings':no_of_ratings,
                            'Age':age,
                            'Publication_Date':publication_date,
                            'Price':price,
                            'Price_Befor':price_befor,
                            'Cover_Type':cover_type,
                            'Link':links,
                            'Description':description,
                            'Product_Details':Product_details,
                           'Best_Seller':best_seller})

In [23]:
# child_books.to_csv('../project-3/data/child_books.csv')

---
## Data dictionary

|Feature|Description|
|---|---|
|name            | the name of the book|
|Series          | what's the Series books that book is part of it|
|Description     | the description of the book|
|Author          | the Author of the book|
|Age             | Target age category|
|Ratingoutof5    | the rating of the book out of 5 |
|NoofRatings     | how many people give there feedback| 
|Price           | the price of the book |
|PriceBefor      | if the book has descend what's it's price before descend|
|CoverType       | the type of book cover |
|PublicationDate | what's the date of publishing the book|
|ProductDetails  | more details about the book |
|BestSeller      | if the book has gotthe best seller|
|Link            | the link of the book|

---
## Clean missing values

In [5]:
data=pd.read_csv('data/child_books.csv')

In [6]:
data.shape

(1200, 15)

In [7]:
data.head()

,Unnamed: 0,Name,Series,Author,Rating,No_of_Ratings,Age,Publication_Date,Price,Price_Befor,Cover_Type,Link,Description,Product_Details,Best_Seller
0,0,We Were Liars,NaN,E. Lockhart,4.5 out of 5 stars,"12,959",Ages: 12 years and up,"May 29, 2018",$5.99,$10.99,Paperback,https://www.amazon.com/We-Were-Liars-Lockhart/...,"\n#1 NEW YORK TIMES BESTSELLER • A modern, sop...",\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,NaN
1,1,Harry Potter Paperback Box Set (Books 1-7),NaN,J. K. Rowling,4.9 out of 5 stars,"34,849",Ages: 8 years and up,"Jul 1, 2009",$50.00,$86.93,Paperback,https://www.amazon.com/Harry-Potter-Paperback-...,"\nNow for the first time ever, J.K. Rowling’s ...",\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,NaN
2,2,If Animals Kissed Good Night,NaN,Ann Whitford Paul,4.8 out of 5 stars,"34,548",Ages: 3 - 6 years,"Jun 3, 2014",$4.79,$7.99,Board book,https://www.amazon.com/If-Animals-Kissed-Good-...,\nDon't miss the other books in this adorable ...,\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,in Poetry for Early Learning
3,3,My First Learn to Write Workbook: Practice for...,NaN,Crystal Radke,4.8 out of 5 stars,"41,827",Ages: 3 - 5 years,"Aug 27, 2019",$5.39,$8.99,Paperback,https://www.amazon.com/My-First-Learn-Write-Wo...,\nA lifetime of communication skills starts he...,\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,in Language Arts Teaching Materials
4,4,I Love You to the Moon and Back,NaN,Amelia Hepworth,4.9 out of 5 stars,"20,949",Ages: 2 - 5 years,"Mar 3, 2015",NaN,NaN,Board book,https://www.amazon.com/I-Love-You-Moon-Back/dp...,\nShow your child just how strong your love is...,\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,in Children's Bear Books


In [8]:
data.columns

Index(['Unnamed: 0', 'Name', 'Series', 'Author', 'Rating', 'No_of_Ratings',
       'Age', 'Publication_Date', 'Price', 'Price_Befor', 'Cover_Type', 'Link',
       'Description', 'Product_Details', 'Best_Seller'],
      dtype='object')

In [9]:
data.drop('Unnamed: 0' ,axis=1 , inplace=True)

In [11]:
data.head()

,Name,Series,Author,Rating,No_of_Ratings,Age,Publication_Date,Price,Price_Befor,Cover_Type,Link,Description,Product_Details,Best_Seller
0,We Were Liars,NaN,E. Lockhart,4.5 out of 5 stars,"12,959",Ages: 12 years and up,"May 29, 2018",$5.99,$10.99,Paperback,https://www.amazon.com/We-Were-Liars-Lockhart/...,"\n#1 NEW YORK TIMES BESTSELLER • A modern, sop...",\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,NaN
1,Harry Potter Paperback Box Set (Books 1-7),NaN,J. K. Rowling,4.9 out of 5 stars,"34,849",Ages: 8 years and up,"Jul 1, 2009",$50.00,$86.93,Paperback,https://www.amazon.com/Harry-Potter-Paperback-...,"\nNow for the first time ever, J.K. Rowling’s ...",\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,NaN
2,If Animals Kissed Good Night,NaN,Ann Whitford Paul,4.8 out of 5 stars,"34,548",Ages: 3 - 6 years,"Jun 3, 2014",$4.79,$7.99,Board book,https://www.amazon.com/If-Animals-Kissed-Good-...,\nDon't miss the other books in this adorable ...,\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,in Poetry for Early Learning
3,My First Learn to Write Workbook: Practice for...,NaN,Crystal Radke,4.8 out of 5 stars,"41,827",Ages: 3 - 5 years,"Aug 27, 2019",$5.39,$8.99,Paperback,https://www.amazon.com/My-First-Learn-Write-Wo...,\nA lifetime of communication skills starts he...,\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,in Language Arts Teaching Materials
4,I Love You to the Moon and Back,NaN,Amelia Hepworth,4.9 out of 5 stars,"20,949",Ages: 2 - 5 years,"Mar 3, 2015",NaN,NaN,Board book,https://www.amazon.com/I-Love-You-Moon-Back/dp...,\nShow your child just how strong your love is...,\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,in Children's Bear Books
5,Superheroes Are Everywhere,NaN,Kamala Harris,4.8 out of 5 stars,"1,726",Ages: 3 - 7 years,"Jan 8, 2019",$14.62,$17.99,Hardcover,https://www.amazon.com/Superheroes-Are-Everywh...,\nFrom Vice President Kamala Harris comes a pi...,\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,in Children's Women Biographies
6,Bronco and Friends: A Party to Remember,NaN,Tim Tebow,4.9 out of 5 stars,381,Ages: 3 - 7 years,"Jan 5, 2021",$12.07,$17.99,Hardcover,https://www.amazon.com/Bronco-Friends-Remember...,\nNEW YORK TIMES BESTSELLER • Heisman Trophy w...,\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,in Children's Christian Family Fiction
7,My First Library : Boxset of 10 Board Books fo...,NaN,Wonder House Books,4.6 out of 5 stars,"12,762",NaN,"Apr 25, 2018",$15.81,$19.99,Board book,https://www.amazon.com/My-First-Library-Boxset...,\nThe amazing boxset contains a set of 10 well...,\n\n\n\nProduct details\n\n\n\n\n\nASIN\n:\n\n...,in Children’s Books about Libraries & Reading
8,Grumpy Monkey,Part of: Grumpy Monkey (4 Books),Suzanne Lang,4.8 out of 5 stars,"9,985",Ages: 3 - 7 years,"May 15, 2018",$10.99,NaN,Hardcover,https://www.amazon.com/Grumpy-Monkey-Suzanne-L...,\nThe hilarious #1 New York Times bestselling ...,\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,in Children's Ape & Monkey Books
9,The Very Hungry Caterpillar,Part of: Rise and Shine (3 Books),Eric Carle,4.9 out of 5 stars,"33,167",Ages: 0 - 3 years,"Mar 23, 1994",$6.99,$10.99,Board book,https://www.amazon.com/Very-Hungry-Caterpillar...,"\nTHE all-time classic picture book, from gene...",\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,in Children's Bug & Spider Books


In [44]:
data.isnull().sum()

Name                  0
Series              820
Author                0
Rating                0
No_of_Ratings         0
Age                  79
Publication_Date     19
Price               143
Price_Befor         327
Cover_Type            0
Link                  0
Description           8
Product_Details      24
Best_Seller         934
dtype: int64

In [41]:
data[data['Price'].isnull()].head(20)

,Name,Series,Author,Rating,No_of_Ratings,Age,Publication_Date,Price,Price_Befor,Cover_Type,Link,Description,Product_Details,Best_Seller
135,Scissor Skills Preschool Workbook for Kids: A ...,NaN,Modern Kid Press,4.7 out of 5 stars,"7,068",NaN,"May 16, 2019",$5.99,NaN,Paperback,https://www.amazon.com/Scissor-Skills-Preschoo...,NaN,\n\n\n\nProduct details\n\n\n\n\n\nPublisher\n...,in Children's Jewelry Crafts
583,Beginner Guitar Lessons for Kids Book: with On...,NaN,Jay Wamsted,4.6 out of 5 stars,907,Ages: 8 - 12 years,"Nov 20, 2019",$9.99,$9.99,Paperback,https://www.amazon.com/Beginner-Guitar-Lessons...,NaN,NaN,NaN
593,The Westing Game (Puffin Modern Classics),Part of: Puffin Modern Classics (12 Books),Ellen Raskin,4.6 out of 5 stars,"2,642",Ages: 8 years and up,"Apr 12, 2004",$0.00,$28.00,Paperback,https://www.amazon.com/Westing-Game-Puffin-Mod...,NaN,NaN,NaN
652,Cat's Cradle Book Kit-,NaN,$9.95,4.7 out of 5 stars,"2,611",Ages: 8 years and up,NaN,$9.99,NaN,Spiral-bound,\n\n\n\nAbout this item\n\n\n\n\nThis fits you...,NaN,NaN,NaN
745,The Cool Bean,Book 3 of 4: The Bad Seed,Jory John,4.9 out of 5 stars,"3,044",Ages: 4 - 8 years,"Dec 3, 2019",$9.48,$18.99,Hardcover,https://www.amazon.com/Cool-Bean-Jory-John/dp/...,NaN,NaN,NaN
896,The Day the Crayons Came Home,NaN,Drew Daywalt,4.8 out of 5 stars,"3,401",Ages: 5 - 8 years,"Aug 18, 2015",$11.75,$18.99,Hardcover,https://www.amazon.com/Day-Crayons-Came-Home/d...,NaN,NaN,NaN
1041,Say Something,NaN,Peter H. Reynolds,4.9 out of 5 stars,"1,054",Ages: 4 - 8 years,"Feb 26, 2019",$9.69,$17.99,Hardcover,https://www.amazon.com/Say-Something-Peter-H-R...,NaN,NaN,NaN
1143,Percy Jackson X 5 Book Set Series Collection 5...,NaN,$4.49,4.8 out of 5 stars,"1,068",NaN,NaN,$27.79,NaN,Paperback,https://www.amazon.com/Percy-Jackson-Book-Set-...,NaN,NaN,NaN


In [43]:
j=data[data['Price'].isnull()].index
print(j)
for r in range(len(j)):
    driver.get(data['Link'][j[r]])
    sleep(5) 
    html = driver.page_source
    soup1 = BeautifulSoup(html,'html')
    print(j[r])
    try:
        try:
            h=soup1.find('div', attrs={ 'id':"tmmSwatches" ,'class':"a-row"}).find('ul', class_="a-unordered-list a-nostyle a-button-list a-horizontal").find('li' ,class_="swatchElement selected resizedSwatchElement").find('span' , class_="a-color-base").text.replace('\n','').replace('from','')
        except:
            h=soup1.find('span', attrs={'class':"a-size-base a-color-price a-color-price"}).text.replace('\n','').replace('from','')
    except:
        h=np.nan
    data['Price'][j[r]]=h

In [45]:
data.isnull().sum()

Name                  0
Series              820
Author                0
Rating                0
No_of_Ratings         0
Age                  79
Publication_Date     19
Price               143
Price_Befor         327
Cover_Type            0
Link                  0
Description           8
Product_Details      24
Best_Seller         934
dtype: int64

In [ ]:
j=data[data['Product_Details'].isnull()].index
print(j)
for r in range(len(j)):
    driver.get(data['Link'][j[r]])
    sleep(5) 
    html = driver.page_source
    soup1 = BeautifulSoup(html,'html')
    print(j[r])
    try:
        h=soup1.find('div', attrs={ 'cel_widget_id':"audible-product-details-btf_csm_instrumentation_wrapper" ,'class':"celwidget"}).find('div',attrs= {'id':"audibleproductdetails_feature_div" ,'class':"celwidget", 'data-feature-name':"audibleproductdetails"}).text#.replace('\n','').replace('from','')
    except:
        h=np.nan
    data['Product_Details'][j[r]]=h

In [ ]:
data.isnull().sum()

In [ ]:
# data.to_csv('../project-3/data/data.csv')

----
## Fix incorrect data types

In [ ]:
df=pd.read_csv('data/data.csv')

In [73]:
df.dtypes

Name                object
Series              object
Author              object
Rating              object
No_of_Ratings       object
Age                 object
Publication_Date    object
Price               object
Price_Befor         object
Cover_Type          object
Link                object
Description         object
Product_Details     object
Best_Seller         object
dtype: object

In [105]:
df['Rating_out_of_5']=df['Rating'].apply(lambda x :float(re.findall(r'\d\.\d', x)[0]))

In [106]:
df.drop(['Unnamed: 0','Rating'],axis=1 , inplace=True)

In [107]:
df['Rating_out_of_5']

0       4.5
1       4.9
2       4.8
3       4.8
4       4.9
       ... 
1195    4.7
1196    4.8
1197    4.9
1198    4.7
1199    4.9
Name: Rating_out_of_5, Length: 1200, dtype: float64

In [108]:
df['No_of_Ratings']=df['No_of_Ratings'].apply(lambda x :float(x.replace(',','.')))

In [109]:
df.dtypes

Name                 object
Series               object
Author               object
No_of_Ratings       float64
Age                  object
Publication_Date     object
Price                object
Price_Befor          object
Cover_Type           object
Link                 object
Description          object
Product_Details      object
Best_Seller          object
Rating_out_of_5     float64
dtype: object

In [110]:
df['Publication_Date']=pd.to_datetime(df['Publication_Date'] )

In [111]:
df.dtypes

Name                        object
Series                      object
Author                      object
No_of_Ratings              float64
Age                         object
Publication_Date    datetime64[ns]
Price                       object
Price_Befor                 object
Cover_Type                  object
Link                        object
Description                 object
Product_Details             object
Best_Seller                 object
Rating_out_of_5            float64
dtype: object

---
## saving the data and upload it on kaggel

In [ ]:
# df.to_csv('../project-3/data/data_books.csv')

### kaggle : [Children books on amazon](https://www.kaggle.com/modhiibrahimalmannaa/1000-children-books-on-amazom)

---
## Conclusions

In this project i have been trying to collecting and cleaning and fixing my own data and upload my first dataset on kaggle